# Initial Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import f1_score, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SelectFromModel
from collections import defaultdict
from imblearn.over_sampling import SVMSMOTE
from imblearn.under_sampling import RandomUnderSampler
import scipy.stats as stats

# Preprocessing

In [3]:
# Import dataset
df = pd.read_csv("./../../Datasets/kieranFeatures_1-31_21-Jan-2025_avgof3.csv").replace([np.inf, -np.inf], np.nan)

df["Lv_1_Lo"] = (df["adjSA1"] < np.percentile(df["adjSA1"], 36)).astype(np.bool_)
df["Lv_2_Lo"] = (df["adjSA2"] < np.percentile(df["adjSA2"], 47)).astype(np.bool_)
df["Lv_3_Lo"] = (df["adjSA3"] < np.percentile(df["adjSA3"], 40)).astype(np.bool_)
df["Tot_Lo"] = (df["adjSAtotal"] < np.percentile(df["adjSAtotal"], 49)).astype(np.bool_)

# Impute missing values with mean of column
for col in df.columns:
    df[col] = df[col].fillna(value = df[col].mean())

# Split up dataset
ids = df["ID"].astype(np.uint8)
# trial_nums = df["trialNum"].astype(np.uint8)
predictors_df = df.drop(columns = ["ID", "trialNum", "adjSA1", "adjSA2", "adjSA3", "adjSAtotal", "Lv_1_Lo", "Lv_2_Lo", "Lv_3_Lo", "Tot_Lo"]).astype(np.float64)
outcomes_df = df[["Lv_1_Lo", "Lv_2_Lo", "Lv_3_Lo", "Tot_Lo"]]
outcomes_df_shuffled = outcomes_df.copy()

# Shuffle labels for shuffled data
outcomes_df_shuffled["Lv_1_Lo"] = np.random.permutation(outcomes_df_shuffled["Lv_1_Lo"])
outcomes_df_shuffled["Lv_2_Lo"] = np.random.permutation(outcomes_df_shuffled["Lv_2_Lo"])
outcomes_df_shuffled["Lv_3_Lo"] = np.random.permutation(outcomes_df_shuffled["Lv_3_Lo"])
outcomes_df_shuffled["Tot_Lo"] = np.random.permutation(outcomes_df_shuffled["Tot_Lo"])

# Free up memory
del col, df

In [18]:
df.iloc[:, 781]

0          -1.84785
1        252.438729
2          4.902291
3          0.622102
4           0.36862
           ...     
295    -3.402921835
296     5.298826105
297    -3.224659795
298    -2.598417765
299     0.008975581
Name: EEG_frpcv_o1_version11, Length: 300, dtype: object